# Analisi dei dati raccolti

## Setup

In [38]:
import pandas as pd

In [39]:
def print_data(series: pd.Series) -> None:
    print(series.value_counts())
    print()
    print(f"Massimo: {series.max()}\nMinimo: {series.min()}\nMedia: {series.mean()}\nVarianza: {series.var()}")

## Asconxofv12

In [40]:
path = "../asconxofv12/data/{}.csv"

### Armv7m

In [41]:
with open(path.format("armv7m"), "r") as f:
    axof_armv7m = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [42]:
# Generazione hash
data = axof_armv7m["0B"]

print_data(data)

0B
86    917
87     79
85      4
Name: count, dtype: int64

Massimo: 87
Minimo: 85
Media: 86.075
Varianza: 0.07745245245245246


#### PT 8 byte

In [43]:
# Generazione hash
data = axof_armv7m["8B"]

print_data(data)

8B
106    875
107    121
105      4
Name: count, dtype: int64

Massimo: 107
Minimo: 105
Media: 106.117
Varianza: 0.11142242242242241


#### PT 16 byte

In [44]:
# Generazione hash
data = axof_armv7m["16B"]

print_data(data)

16B
125    759
127    121
126    120
Name: count, dtype: int64

Massimo: 127
Minimo: 125
Media: 125.362
Varianza: 0.4734294294294295


#### PT 32 byte

In [45]:
# Generazione hash
data = axof_armv7m["32B"]

print_data(data)

32B
165    838
166    161
167      1
Name: count, dtype: int64

Massimo: 167
Minimo: 165
Media: 165.163
Varianza: 0.13856956956956956


#### PT 64 byte

In [46]:
# Generazione hash
data = axof_armv7m["64B"]

print_data(data)

64B
244    760
245    239
243      1
Name: count, dtype: int64

Massimo: 245
Minimo: 243
Media: 244.238
Varianza: 0.18353953953953953


#### PT 128 byte

In [47]:
# Generazione hash
data = axof_armv7m["128B"]

print_data(data)

128B
402    479
404    400
403    120
405      1
Name: count, dtype: int64

Massimo: 405
Minimo: 402
Media: 402.923
Varianza: 0.8779489489489491


#### PT 256 byte

In [48]:
# Generazione hash
data = axof_armv7m["256B"]

print_data(data)

256B
720    719
719    281
Name: count, dtype: int64

Massimo: 720
Minimo: 719
Media: 719.719
Varianza: 0.20224124124124118


#### PT 512 byte

In [49]:
# Generazione hash
data = axof_armv7m["512B"]

print_data(data)

512B
1354    398
1356    361
1355    241
Name: count, dtype: int64

Massimo: 1356
Minimo: 1354
Media: 1354.963
Varianza: 0.7583893893893893


#### PT 1024 byte

In [50]:
# Generazione hash
data = axof_armv7m["1024B"]

print_data(data)

1024B
2624    640
2623    358
2626      2
Name: count, dtype: int64

Massimo: 2626
Minimo: 2623
Media: 2623.646
Varianza: 0.240924924924925


### Armv7m_lowsize

In [51]:
with open(path.format("armv7m_lowsize"), "r") as f:
    axof_lowsize = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [52]:
# Generazione hash
data = axof_lowsize["0B"]

print_data(data)

0B
70    917
71     43
72     40
Name: count, dtype: int64

Massimo: 72
Minimo: 70
Media: 70.123
Varianza: 0.18805905905905906


#### PT 8 byte

In [53]:
# Generazione hash
data = axof_lowsize["8B"]

print_data(data)

8B
87    880
88    120
Name: count, dtype: int64

Massimo: 88
Minimo: 87
Media: 87.12
Varianza: 0.1057057057057057


#### PT 16 byte

In [54]:
# Generazione hash
data = axof_lowsize["16B"]

print_data(data)

16B
103    839
105     81
104     80
Name: count, dtype: int64

Massimo: 105
Minimo: 103
Media: 103.242
Varianza: 0.3457817817817818


#### PT 32 byte

In [55]:
# Generazione hash
data = axof_lowsize["32B"]

print_data(data)

32B
136    758
138    160
137     82
Name: count, dtype: int64

Massimo: 138
Minimo: 136
Media: 136.402
Varianza: 0.5609569569569569


#### PT 64 byte

In [56]:
# Generazione hash
data = axof_lowsize["64B"]

print_data(data)

64B
203    760
204    198
202     42
Name: count, dtype: int64

Massimo: 204
Minimo: 202
Media: 203.156
Varianza: 0.21587987987987983


#### PT 128 byte

In [57]:
# Generazione hash
data = axof_lowsize["128B"]

print_data(data)

128B
335    637
336    361
338      1
337      1
Name: count, dtype: int64

Massimo: 338
Minimo: 335
Media: 335.366
Varianza: 0.2402842842842843


#### PT 256 byte

In [58]:
# Generazione hash
data = axof_lowsize["256B"]

print_data(data)

256B
601    600
600    399
602      1
Name: count, dtype: int64

Massimo: 602
Minimo: 600
Media: 600.602
Varianza: 0.24183783783783785


#### PT 512 byte

In [59]:
# Generazione hash
data = axof_lowsize["512B"]

print_data(data)

512B
1132    801
1133    159
1131     40
Name: count, dtype: int64

Massimo: 1133
Minimo: 1131
Media: 1132.119
Varianza: 0.18502402402402401


#### PT 1024 byte

In [60]:
# Generazione hash
data = axof_lowsize["1024B"]

print_data(data)

1024B
2193    798
2194    201
2195      1
Name: count, dtype: int64

Massimo: 2195
Minimo: 2193
Media: 2193.203
Varianza: 0.1639549549549549


### Armv7m_small

In [61]:
with open(path.format("armv7m_small"), "r") as f:
    axof_small = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [62]:
# Generazione hash
data = axof_small["0B"]

print_data(data)

0B
70    918
71     81
72      1
Name: count, dtype: int64

Massimo: 72
Minimo: 70
Media: 70.083
Varianza: 0.07818918918918918


#### PT 8 byte

In [63]:
# Generazione hash
data = axof_small["8B"]

print_data(data)

8B
87    880
88    120
Name: count, dtype: int64

Massimo: 88
Minimo: 87
Media: 87.12
Varianza: 0.1057057057057057


#### PT 16 byte

In [64]:
# Generazione hash
data = axof_small["16B"]

print_data(data)

16B
103    840
105    121
104     39
Name: count, dtype: int64

Massimo: 105
Minimo: 103
Media: 103.281
Varianza: 0.4444834834834834


#### PT 32 byte

In [65]:
# Generazione hash
data = axof_small["32B"]

print_data(data)

32B
136    479
137    361
138    160
Name: count, dtype: int64

Massimo: 138
Minimo: 136
Media: 136.681
Varianza: 0.5377767767767768


#### PT 64 byte

In [66]:
# Generazione hash
data = axof_small["64B"]

print_data(data)

64B
203    800
204    198
202      2
Name: count, dtype: int64

Massimo: 204
Minimo: 202
Media: 203.196
Varianza: 0.16174574574574574


#### PT 128 byte

In [67]:
# Generazione hash
data = axof_small["128B"]

print_data(data)

128B
335    637
336    361
338      1
337      1
Name: count, dtype: int64

Massimo: 338
Minimo: 335
Media: 335.366
Varianza: 0.2402842842842843


#### PT 256 byte

In [68]:
# Generazione hash
data = axof_small["256B"]

print_data(data)

256B
601    600
600    399
602      1
Name: count, dtype: int64

Massimo: 602
Minimo: 600
Media: 600.602
Varianza: 0.24183783783783785


#### PT 512 byte

In [69]:
# Generazione hash
data = axof_small["512B"]

print_data(data)

512B
1132    840
1133    159
1131      1
Name: count, dtype: int64

Massimo: 1133
Minimo: 1131
Media: 1132.158
Varianza: 0.13517117117117117


#### PT 1024 byte

In [70]:
# Generazione hash
data = axof_small["1024B"]

print_data(data)

1024B
2193    798
2194    201
2195      1
Name: count, dtype: int64

Massimo: 2195
Minimo: 2193
Media: 2193.203
Varianza: 0.1639549549549549


### Bi32_armv7m

In [71]:
with open(path.format("bi32_armv7m"), "r") as f:
    axof_bi32 = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [72]:
# Generazione hash
data = axof_bi32["0B"]

print_data(data)

0B
71    920
72     80
Name: count, dtype: int64

Massimo: 72
Minimo: 71
Media: 71.08
Varianza: 0.0736736736736737


#### PT 8 byte

In [73]:
# Generazione hash
data = axof_bi32["8B"]

print_data(data)

8B
86    917
87     83
Name: count, dtype: int64

Massimo: 87
Minimo: 86
Media: 86.083
Varianza: 0.07618718718718716


#### PT 16 byte

In [74]:
# Generazione hash
data = axof_bi32["16B"]

print_data(data)

16B
102    877
103    121
101      2
Name: count, dtype: int64

Massimo: 103
Minimo: 101
Media: 102.119
Varianza: 0.10894794794794795


#### PT 32 byte

In [75]:
# Generazione hash
data = axof_bi32["32B"]

print_data(data)

32B
132    880
133    120
Name: count, dtype: int64

Massimo: 133
Minimo: 132
Media: 132.12
Varianza: 0.1057057057057057


#### PT 64 byte

In [76]:
# Generazione hash
data = axof_bi32["64B"]

print_data(data)

64B
193    799
194    200
195      1
Name: count, dtype: int64

Massimo: 195
Minimo: 193
Media: 193.202
Varianza: 0.16335935935935933


#### PT 128 byte

In [77]:
# Generazione hash
data = axof_bi32["128B"]

print_data(data)

128B
315    681
316    319
Name: count, dtype: int64

Massimo: 316
Minimo: 315
Media: 315.319
Varianza: 0.21745645645645648


#### PT 256 byte

In [78]:
# Generazione hash
data = axof_bi32["256B"]

print_data(data)

256B
560    558
559    440
562      1
561      1
Name: count, dtype: int64

Massimo: 562
Minimo: 559
Media: 559.563
Varianza: 0.2542852852852852


#### PT 512 byte

In [79]:
# Generazione hash
data = axof_bi32["512B"]

print_data(data)

512B
1049    721
1050    239
1051     40
Name: count, dtype: int64

Massimo: 1051
Minimo: 1049
Media: 1049.319
Varianza: 0.2975365365365366


#### PT 1024 byte

In [80]:
# Generazione hash
data = axof_bi32["1024B"]

print_data(data)

1024B
2028    960
2029     40
Name: count, dtype: int64

Massimo: 2029
Minimo: 2028
Media: 2028.04
Varianza: 0.03843843843843842


### Ref

In [81]:
with open(path.format("ref"), "r") as f:
    axof_ref = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [82]:
# Generazione hash
data = axof_ref["0B"]

print_data(data)

0B
298    599
299    242
300    159
Name: count, dtype: int64

Massimo: 300
Minimo: 298
Media: 298.56
Varianza: 0.5649649649649651


#### PT 8 byte

In [83]:
# Generazione hash
data = axof_ref["8B"]

print_data(data)

8B
359    398
361    361
360    240
362      1
Name: count, dtype: int64

Massimo: 362
Minimo: 359
Media: 359.965
Varianza: 0.7625375375375375


#### PT 16 byte

In [84]:
# Generazione hash
data = axof_ref["16B"]

print_data(data)

16B
421    598
422    401
424      1
Name: count, dtype: int64

Massimo: 424
Minimo: 421
Media: 421.404
Varianza: 0.24703103103103108


#### PT 32 byte

In [85]:
# Generazione hash
data = axof_ref["32B"]

print_data(data)

32B
545    519
543    240
544    240
546      1
Name: count, dtype: int64

Massimo: 546
Minimo: 543
Media: 544.281
Varianza: 0.6847237237237236


#### PT 64 byte

In [86]:
# Generazione hash
data = axof_ref["64B"]

print_data(data)

64B
790    758
789    239
791      3
Name: count, dtype: int64

Massimo: 791
Minimo: 789
Media: 789.764
Varianza: 0.18649049049049052


#### PT 128 byte

In [87]:
# Generazione hash
data = axof_ref["128B"]

print_data(data)

128B
1282    719
1283    280
1284      1
Name: count, dtype: int64

Massimo: 1284
Minimo: 1282
Media: 1282.282
Varianza: 0.20468068068068068


#### PT 256 byte

In [88]:
# Generazione hash
data = axof_ref["256B"]

print_data(data)

256B
2266    600
2267    241
2265    159
Name: count, dtype: int64

Massimo: 2267
Minimo: 2265
Media: 2266.082
Varianza: 0.39366966966966976


#### PT 512 byte

In [89]:
# Generazione hash
data = axof_ref["512B"]

print_data(data)

512B
4233    800
4234    200
Name: count, dtype: int64

Massimo: 4234
Minimo: 4233
Media: 4233.2
Varianza: 0.1601601601601601


#### PT 1024 byte

In [90]:
# Generazione hash
data = axof_ref["1024B"]

print_data(data)

1024B
8168    841
8169    159
Name: count, dtype: int64

Massimo: 8169
Minimo: 8168
Media: 8168.159
Varianza: 0.13385285285285287
